In [28]:
import xarray as xr
import cfgrib
import numpy as np
import optuna

from functools import partial
from sklearn.metrics import mean_squared_error

# import matplotlib.pyplot as plt

from data_processor import DataProcessor
from regressor import Regressor

In [29]:
grib_data = cfgrib.open_datasets('../data2022.grib')
surface = grib_data[0] 
hybrid = grib_data[1] 

In [30]:
feature_list = ['t2m', 'sp', 'tcc', 'u10', 'v10', 'tp']
t2m = surface.t2m.to_numpy() - 273.15  # -> C
sp = surface.sp.to_numpy() / 100       # -> hPa
tcc = surface.tcc.to_numpy()
u10 = surface.u10.to_numpy()
v10 = surface.v10.to_numpy()
tp = hybrid.tp.to_numpy().reshape((-1,) + hybrid.tp.shape[2:])
data = np.stack((t2m, sp, tcc, u10, v10, tp), axis=-1)

In [31]:
def objective(trial, data, feature_list):
    #Parameters for optimization
    s = trial.suggest_int('s', 3, 5)  
    fh = trial.suggest_int('fh', 1, 3) 
    regressor_type = trial.suggest_categorical('regressor_type', ['linear', 'lasso', 'ridge', 'elastic_net'])


    processor = DataProcessor(data)
    X, y = processor.preprocess(s, fh, use_neighbours=False)
    X_train, X_test, y_train, y_test = processor.train_test_split(X, y)
    
    
    linearreg = Regressor(X.shape, fh, feature_list, regressor_type=regressor_type)
    linearreg.train(X_train, y_train)
    y_hat = linearreg.predict_and_evaluate(X_test, y_test, max_samples=0)
    
    #returns a MSE
    #TODO RMSE or regressor method calculating MSE
    return mean_squared_error(y_test.reshape(-1,1), y_hat.reshape(-1,1))

In [32]:
#create an optuna study and objective function
study = optuna.create_study(direction='minimize')
objective_func = partial(objective, data=data, feature_list=feature_list)

[I 2023-11-05 23:30:06,079] A new study created in memory with name: no-name-0c555f6e-ecd6-4dc9-9b2f-ec86bf3ee2b3


In [33]:
n_trials = 100  # Number of trials to run
study.optimize(objective_func, n_trials=n_trials)

[I 2023-11-05 23:30:25,222] Trial 0 finished with value: 1.8219843858646139 and parameters: {'s': 5, 'fh': 1, 'regressor_type': 'lasso'}. Best is trial 0 with value: 1.8219843858646139.


Evaluation metrics for entire test set:
t2m => RMSE: 2.037; SE: 0.010200797752953046
sp => RMSE: 1.564; SE: 0.03702956939722691
tcc => RMSE: 0.355; SE: 0.0004828579478474195
u10 => RMSE: 1.481; SE: 0.005026088934293758
v10 => RMSE: 1.42; SE: 0.004110117665590181
tp => RMSE: 0.0; SE: 3.4696299529253065e-07


[I 2023-11-05 23:30:28,571] Trial 1 finished with value: 3.6258206090457614 and parameters: {'s': 3, 'fh': 3, 'regressor_type': 'ridge'}. Best is trial 0 with value: 1.8219843858646139.


Evaluation metrics for entire test set:
t2m => RMSE: 2.621; SE: 0.005901756074861131
sp => RMSE: 2.766; SE: 0.02138352719199003
tcc => RMSE: 0.288; SE: 0.0002772875764415203
u10 => RMSE: 2.054; SE: 0.002905309043208137
v10 => RMSE: 1.712; SE: 0.0023799655932334822
tp => RMSE: 0.0; SE: 2.0052221260247116e-07


[I 2023-11-05 23:30:50,741] Trial 2 finished with value: 2.916478036916376 and parameters: {'s': 4, 'fh': 2, 'regressor_type': 'elastic_net'}. Best is trial 0 with value: 1.8219843858646139.


...

Evaluation metrics for entire test set:
t2m => RMSE: 1.9; SE: 0.010200797752953046
sp => RMSE: 1.179; SE: 0.03702956939722691
tcc => RMSE: 0.256; SE: 0.0004828579478474195
u10 => RMSE: 1.44; SE: 0.005026088934293758
v10 => RMSE: 1.216; SE: 0.004110117665590181
tp => RMSE: 0.0; SE: 3.4696299529253065e-07


[I 2023-11-05 23:48:07,922] Trial 98 finished with value: 1.436537361466356 and parameters: {'s': 5, 'fh': 1, 'regressor_type': 'linear'}. Best is trial 46 with value: 1.436537361466356.


Evaluation metrics for entire test set:
t2m => RMSE: 1.9; SE: 0.010200797752953046
sp => RMSE: 1.179; SE: 0.03702956939722691
tcc => RMSE: 0.256; SE: 0.0004828579478474195
u10 => RMSE: 1.44; SE: 0.005026088934293758
v10 => RMSE: 1.216; SE: 0.004110117665590181
tp => RMSE: 0.0; SE: 3.4696299529253065e-07


[I 2023-11-05 23:48:20,719] Trial 99 finished with value: 1.436537361466356 and parameters: {'s': 5, 'fh': 1, 'regressor_type': 'linear'}. Best is trial 46 with value: 1.436537361466356.


Evaluation metrics for entire test set:
t2m => RMSE: 1.9; SE: 0.010200797752953046
sp => RMSE: 1.179; SE: 0.03702956939722691
tcc => RMSE: 0.256; SE: 0.0004828579478474195
u10 => RMSE: 1.44; SE: 0.005026088934293758
v10 => RMSE: 1.216; SE: 0.004110117665590181
tp => RMSE: 0.0; SE: 3.4696299529253065e-07


In [34]:
best_s = study.best_params['s']
best_fh = study.best_params['fh']
best_regressor_type = study.best_params['regressor_type']

In [35]:
print(f"Best hyperparameters - s: {best_s}, fh: {best_fh}")
print(f"Best regressor type: {best_regressor_type}")

Best hyperparameters - s: 5, fh: 1
Best regressor type: linear
